Student Name: Joaz Ephraim A. Flores

In [117]:
# Create a new database.
import sqlite3
conn = sqlite3.connect('joazenrollmentsystem.db')

#For purposes of demonstration I did not put a foreign key as it will be difficult to make changes to the tables.

# Create the departments table.
conn = sqlite3.connect('joazenrollmentsystem.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS departments")
query = """CREATE TABLE departments(
           departmentid INTEGER PRIMARY KEY AUTOINCREMENT,
           departmentname VARCHAR(100) UNIQUE NOT NULL,
           departmentcode VARCHAR(10) UNIQUE NOT NULL)"""
cursor.execute(query)
conn.commit()

# Create the professors table.
conn = sqlite3.connect('joazenrollmentsystem.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS professors")
query = """CREATE TABLE professors(
           professorid INTEGER PRIMARY KEY AUTOINCREMENT,
           firstname VARCHAR(50) NOT NULL,
           lastname VARCHAR(50) NOT NULL,
           email VARCHAR(100) UNIQUE NOT NULL,
           phone VARCHAR(15) UNIQUE NOT NULL)"""
cursor.execute(query)
conn.commit()

# Create the students table.
conn = sqlite3.connect('joazenrollmentsystem.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS students")
query = """CREATE TABLE students(
           studentid INTEGER PRIMARY KEY AUTOINCREMENT,
           firstname VARCHAR(50) NOT NULL,
           lastname VARCHAR(50) NOT NULL,
           email VARCHAR(100) UNIQUE NOT NULL,
           phone VARCHAR(15) UNIQUE NOT NULL,
           departmentid INT)"""
           # FOREIGN KEY (departmentid) REFERENCES Departments(departmentid)
cursor.execute(query)
conn.commit()

# Create the courses table.
conn = sqlite3.connect('joazenrollmentsystem.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS courses")
query = """CREATE TABLE courses(
           courseid INTEGER PRIMARY KEY AUTOINCREMENT,
           coursename VARCHAR(100) NOT NULL,
           coursecode VARCHAR(10) UNIQUE NOT NULL,
           departmentid INT)"""
           # FOREIGN KEY (departmentid) REFERENCES Departments(departmentid)
cursor.execute(query)
conn.commit()

# Create the enrollments table.
conn = sqlite3.connect('joazenrollmentsystem.db')
cursor = conn.cursor()
cursor.execute("DROP TABLE IF EXISTS enrollments")
query = """CREATE TABLE enrollments(
           enrollmentid INTEGER PRIMARY KEY AUTOINCREMENT,
           studentid INT,
           courseid INT,
           departmentid INT,
           professorid INT,
           enrollmentdate DATE NOT NULL,
           schoolyear DATE NOT NULL)"""
           # FOREIGN KEY (studentid) REFERENCES Students(studentid),
           # FOREIGN KEY (courseid) REFERENCES Courses(courseid),
           # FOREIGN KEY (departmentid) REFERENCES Departments(departmentid),
           # FOREIGN KEY (professorid) REFERENCES Professors(professorid)
cursor.execute(query)
conn.commit()
conn.close()


In [50]:
import sqlite3
from datetime import date

class EnrollmentSystem2:
    def __init__(self, db_name="joazenrollmentsystem.db"):
        self.db_name = db_name
        self.conn = sqlite3.connect(self.db_name)

    def close(self): # I use this when I occassionally run into threading issues.
        """Explicitly close the database connection."""
        if self.conn:
            self.conn.close()
            self.conn = None  # Mark the connection as closed

    # --- CRUD for Students ---
    def add_student(self, firstname, lastname, email, phone, departmentid=None):
        """Create: Add a new student."""
        with self.conn: # The WITH keyword in sqlite3 acts like a transaction in SQL. If an error happens during an update it gets rolled back.
            self.conn.execute("""
                INSERT INTO Students (firstname, lastname, email, phone, departmentid)
                VALUES (?, ?, ?, ?, ?);
            """, (firstname, lastname, email, phone, departmentid))

    def get_students(self):
        """Read: Retrieve all students."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM Students;")
        return cursor.fetchall()

    def update_student(self, studentid, firstname=None, lastname=None, email=None, phone=None, departmentid=None):
        """Update: Modify a student's information."""
        updates = []
        params = []

        if firstname:
            updates.append("firstname = ?")
            params.append(firstname)
        if lastname:
            updates.append("lastname = ?")
            params.append(lastname)
        if email:
            updates.append("email = ?")
            params.append(email)
        if phone:
            updates.append("phone = ?")
            params.append(phone)
        if departmentid:
            updates.append("departmentid = ?")
            params.append(departmentid)

        params.append(studentid) # Added safety against SQL injections.
        query = f"UPDATE Students SET {', '.join(updates)} WHERE studentid = ?;"

        with self.conn:
            self.conn.execute(query, params)

    def delete_student(self, studentid):
        """Delete: Remove a student with confirmation."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT firstname, lastname FROM Students WHERE studentid = ?;", (studentid,))
        student = cursor.fetchone()

        if student:
            firstname, lastname = student
            confirmation = input(f"Are you sure you want to delete the student '{firstname} {lastname}' (ID: {studentid})? (yes/no): ").strip().lower()
            if confirmation == 'yes':
                with self.conn:
                    self.conn.execute("DELETE FROM Students WHERE studentid = ?;", (studentid,))
                print(f"Student '{firstname} {lastname}' has been deleted.")
            else:
                print("Deletion cancelled.")
        else:
            print(f"No student found with ID: {studentid}")

    # --- CRUD for Professors ---
    def add_professor(self, firstname, lastname, email, phone):
        """Create: Add a new professor."""
        with self.conn:
            self.conn.execute("""
                INSERT INTO Professors (firstname, lastname, email, phone)
                VALUES (?, ?, ?, ?);
            """, (firstname, lastname, email, phone))

    def get_professors(self):
        """Read: Retrieve all professors."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM Professors;")
        return cursor.fetchall()

    def update_professor(self, professorid, firstname=None, lastname=None, email=None, phone=None):
        """Update: Modify a professor's information."""
        updates = []
        params = []

        if firstname:
            updates.append("firstname = ?")
            params.append(firstname)
        if lastname:
            updates.append("lastname = ?")
            params.append(lastname)
        if email:
            updates.append("email = ?")
            params.append(email)
        if phone:
            updates.append("phone = ?")
            params.append(phone)

        params.append(professorid)
        query = f"UPDATE Professors SET {', '.join(updates)} WHERE professorid = ?;"

        with self.conn:
            self.conn.execute(query, params)

    def delete_professor(self, professorid):
        """Delete: Remove a professor with confirmation."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT firstname, lastname FROM Professors WHERE professorid = ?;", (professorid,))
        professor = cursor.fetchone()

        if professor:
            firstname, lastname = professor
            confirmation = input(f"Are you sure you want to delete the professor '{firstname} {lastname}' (ID: {professorid})? (yes/no): ").strip().lower()
            if confirmation == 'yes':
                with self.conn:
                    self.conn.execute("DELETE FROM Professors WHERE professorid = ?;", (professorid,))
                print(f"Professor '{firstname} {lastname}' has been deleted.")
            else:
                print("Deletion cancelled.")
        else:
            print(f"No professor found with ID: {professorid}")

    # --- CRUD for Departments ---
    def add_department(self, departmentname, departmentcode):
        """Create: Add a new department."""
        with self.conn:
            self.conn.execute("""
                INSERT INTO Departments (departmentname, departmentcode)
                VALUES (?, ?);
            """, (departmentname, departmentcode))

    def get_departments(self):
        """Read: Retrieve all departments."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM Departments;")
        return cursor.fetchall()

    def update_department(self, departmentid, departmentname=None, departmentcode=None):
        """Update: Modify a department's information."""
        updates = []
        params = []

        if departmentname:
            updates.append("departmentname = ?")
            params.append(departmentname)
        if departmentcode:
            updates.append("departmentcode = ?")
            params.append(departmentcode)

        params.append(departmentid)
        query = f"UPDATE Departments SET {', '.join(updates)} WHERE departmentid = ?;"

        with self.conn:
            self.conn.execute(query, params)

    def delete_department(self, departmentid):
        """Delete: Remove a department with confirmation."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT departmentname FROM Departments WHERE departmentid = ?;", (departmentid,))
        department = cursor.fetchone()

        if department:
            departmentname = department[0]
            confirmation = input(f"Are you sure you want to delete the department '{departmentname}' (ID: {departmentid})? (yes/no): ").strip().lower()
            if confirmation == 'yes':
                with self.conn:
                    self.conn.execute("DELETE FROM Departments WHERE departmentid = ?;", (departmentid,))
                print(f"Department '{departmentname}' has been deleted.")
            else:
                print("Deletion cancelled.")
        else:
            print(f"No department found with ID: {departmentid}")

    # --- CRUD for Courses ---
    def add_course(self, coursename, coursecode, departmentid):
        """Create: Add a new course."""
        with self.conn:
            self.conn.execute("""
                INSERT INTO Courses (coursename, coursecode, departmentid)
                VALUES (?, ?, ?);
            """, (coursename, coursecode, departmentid))

    def get_courses(self):
        """Read: Retrieve all courses."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT * FROM Courses;")
        return cursor.fetchall()

    def update_course(self, courseid, coursename=None, coursecode=None, departmentid=None):
        """Update: Modify course information."""
        updates = []
        params = []

        if coursename:
            updates.append("coursename = ?")
            params.append(coursename)
        if coursecode:
            updates.append("coursecode = ?")
            params.append(coursecode)
        if departmentid:
            updates.append("departmentid = ?")
            params.append(departmentid)

        params.append(courseid)
        query = f"UPDATE Courses SET {', '.join(updates)} WHERE courseid = ?;"

        with self.conn:
            self.conn.execute(query, params)

    def delete_course(self, courseid):
        """Delete: Remove a course with confirmation."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT coursename FROM Courses WHERE courseid = ?;", (courseid,))
        course = cursor.fetchone()

        if course:
            coursename = course[0]
            confirmation = input(f"Are you sure you want to delete the course '{coursename}' (ID: {courseid})? (yes/no): ").strip().lower()
            if confirmation == 'yes':
                with self.conn:
                    self.conn.execute("DELETE FROM Courses WHERE courseid = ?;", (courseid,))
                print(f"Course '{coursename}' has been deleted.")
            else:
                print("Deletion cancelled.")
        else:
            print(f"No course found with ID: {courseid}")

    # --- CRUD for Enrollments ---
    def enroll_student(self, studentid, courseid, departmentid, professorid, schoolyear):
        """Create: Enroll a student in a course."""
        enrollment_date = date.today()
        with self.conn:
            self.conn.execute("""
                INSERT INTO Enrollments (studentid, courseid, departmentid, professorid, enrollmentdate, schoolyear)
                VALUES (?, ?, ?, ?, ?, ?);
            """, (studentid, courseid, departmentid, professorid, enrollment_date, schoolyear))

    def get_enrollments(self):
        """Read: Retrieve all enrollments."""
        cursor = self.conn.cursor()
        cursor.execute("""
            SELECT
                Enrollments.enrollmentid,
                Students.firstname || ' ' || Students.lastname AS student_name,
                Courses.coursename,
                Departments.departmentname,
                Professors.firstname || ' ' || Professors.lastname AS professor_name,
                Enrollments.enrollmentdate,
                Enrollments.schoolyear
            FROM Enrollments
            INNER JOIN Students ON Enrollments.studentid = Students.studentid
            INNER JOIN Courses ON Enrollments.courseid = Courses.courseid
            INNER JOIN Departments ON Enrollments.departmentid = Departments.departmentid
            INNER JOIN Professors ON Enrollments.professorid = Professors.professorid;
        """)
        return cursor.fetchall()

    def delete_enrollment(self, enrollmentid):
        """Delete: Remove an enrollment with confirmation."""
        cursor = self.conn.cursor()
        cursor.execute("SELECT enrollmentid FROM Enrollments WHERE enrollmentid = ?;", (enrollmentid,))
        enrollment = cursor.fetchone()

        if enrollment:
            confirmation = input(f"Are you sure you want to delete the enrollment (ID: {enrollmentid})? (yes/no): ").strip().lower()
            if confirmation == 'yes':
                with self.conn:
                    self.conn.execute("DELETE FROM Enrollments WHERE enrollmentid = ?;", (enrollmentid,))
                print(f"Enrollment (ID: {enrollmentid}) has been deleted.")
            else:
                print("Deletion cancelled.")
        else:
            print(f"No enrollment found with ID: {enrollmentid}")

    # --- Cleanup --- (I occasionally run into threading issues.)
    def __del__(self):
        """Close the database connection."""
        if self.conn:
            self.conn.close()


In [120]:
# Initialize the system
system = EnrollmentSystem2()


In [119]:
# Close the System
system.close()


CRUD Demonstration Departments

In [121]:
# Add a Department
system.add_department(departmentname="Information Technology", departmentcode="IT")


In [97]:
# Retrieve Departments
departments = system.get_departments()
for department in departments:
    print(department)


In [94]:
# Update a Department
system.update_department(departmentid=1, departmentname="Marketing", departmentcode="MKTG")


In [96]:
# Delete a Department
system.delete_department(departmentid=1)


Are you sure you want to delete the department 'Marketing' (ID: 1)? (yes/no): yes
Department 'Marketing' has been deleted.


CRUD Demonstration Professors

In [122]:
# Add a Professor
system.add_professor(firstname="Juan", lastname="Delacruz", email="juan.delacruz@tip.edu.ph", phone="987-654-3210")


In [104]:
# Retrieve Professors
professors = system.get_professors()
for professor in professors:
    print(professor)


In [100]:
# Update a Professor
system.update_professor(professorid=1, lastname="Marquez", email="juan.marquez@tip.edu.ph", phone="111-222-3333")


In [103]:
# Delete a Professor
system.delete_professor(professorid=1)


Are you sure you want to delete the professor 'Juan Marquez' (ID: 1)? (yes/no): yes
Professor 'Juan Marquez' has been deleted.


CRUD Demonstration Courses

In [123]:
# Add a Course
system.add_course(coursename="Data Structures", coursecode="CS101", departmentid=1)


In [110]:
# Retrieve Courses
courses = system.get_courses()
for course in courses:
    print(course)


In [107]:
# Update a Course
system.update_course(courseid=1, coursename="Advanced Data Structures")


In [109]:
# Delete a Course
system.delete_course(courseid=1)


Are you sure you want to delete the course 'Advanced Data Structures' (ID: 1)? (yes/no): yes
Course 'Advanced Data Structures' has been deleted.


CRUD Demonstration Students

In [124]:
# Add a student
system.add_student(firstname="John", lastname="Doe", email="john.doe@yahoo.com", phone="123-456-7890", departmentid=1)


In [116]:
# Retrieve Students
students = system.get_students()
for student in students:
    print(student)


In [113]:
# Update a Student
system.update_student(studentid=1, firstname="Jonathan", email="jon.doe@yahoo.com")


In [115]:
# Delete a Student
system.delete_student(studentid=1)


Are you sure you want to delete the student 'Jonathan Doe' (ID: 1)? (yes/no): yes
Student 'Jonathan Doe' has been deleted.


CRUD Demonstration Enrollments

In [125]:
# Enroll a Student
system.enroll_student(studentid=1, courseid=1, departmentid=1, professorid=1, schoolyear="2024-2025")


In [128]:
# Retrieve Enrollments
enrollments = system.get_enrollments()
for enrollment in enrollments:
    print(enrollment)


In [127]:
# Delete an Enrollment
system.delete_enrollment(enrollmentid=1)


Are you sure you want to delete the enrollment (ID: 1)? (yes/no): yes
Enrollment (ID: 1) has been deleted.
